In [1]:
import pandas as pd
import numpy as np

ww = pd.read_csv('C:\Asus WebStorage\psabin@gmail.com\MySyncFolder\Data Science Course\BlueBerry Winery\winequality-white.csv', sep=';')
rw = pd.read_csv('C:\Asus WebStorage\psabin@gmail.com\MySyncFolder\Data Science Course\BlueBerry Winery\winequality-red.csv', sep=';')

In [2]:
ww['quality_label'] = ww['quality'].apply(lambda value: 'low'
if value <= 5 else 'medium'
if value <= 7 else 'high')

# here we are transforming these labels into categrical data type (specific to pandas) instead of simple string
ww['quality_label'] = pd.Categorical(ww['quality_label'],
categories=['low', 'medium', 'high'])

rw['quality_label'] = rw['quality'].apply(lambda value: 'low'
if value <= 5 else 'medium'
if value <= 7 else 'high')

# here we are transforming these labels into categrical data type (specific to pandas) instead of simple string
rw['quality_label'] = pd.Categorical(rw['quality_label'],
categories=['low', 'medium', 'high'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
ww['encoded_quality_label'] = le.fit_transform(ww['quality_label'])  


In [ ]:
# Swap 0 for 3 to make the order make sense from low to high
ww['encoded_quality_label'].replace(0,3,inplace=True)
display(ww.head(20))

In [ ]:
ww.drop(['quality_label'], axis=1, inplace=True)

In [7]:
ww.drop(['quality'], axis=1, inplace=True)
display(ww.head())

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,encoded_quality_label
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,2
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,2
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,2
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,2
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,2


## Select the features to use as inputs

In [17]:
ww_select = ww[['fixed acidity', 'volatile acidity', 'chlorides', 'alcohol']]
display(ww_select.head())

,fixed acidity,volatile acidity,chlorides,alcohol
0,7.0,0.27,0.045,8.8
1,6.3,0.30,0.049,9.5
2,8.1,0.28,0.050,10.1
3,7.2,0.23,0.058,9.9
4,7.2,0.23,0.058,9.9


## Create column-vector for quality_label (y)

In [20]:
y_ww_quality = ww.encoded_quality_label
display(y_ww_quality.head())
display(y_ww_quality.shape)

0    2
1    2
2    2
3    2
4    2
Name: encoded_quality_label, dtype: int32

(4898,)

## Split into train and test data

In [44]:
from sklearn.model_selection import train_test_split
ww_features_train, ww_features_test, ww_quality_train, ww_quality_test = train_test_split(ww_select,y_ww_quality,test_size=0.2, random_state=19)

## Scale wine features to even out their impact on the model

In [45]:

# Also try different methods of normalization/standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
ww_features_train_scaled = scaler.fit_transform(ww_features_train)
ww_features_test_scaled = scaler.transform(ww_features_test)

## Initialize and fit the model

In [25]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(ww_features_train_scaled, ww_quality_train)

LogisticRegression(max_iter=1000)

## Predict the test set

In [26]:
pred_model = model.predict(ww_features_test_scaled)

## Show accuracy score

In [38]:
from sklearn.metrics import accuracy_score
accuracy = ((accuracy_score(ww_quality_test, pred_model)) * 100).round(2)

print(f"{accuracy}% accuracy with prediction of low, medium, or high quality of a given wine, based on fixed acidity, volatile acidity, \nchlorides, and alcohol content.")


68.57

68.57% accuracy with prediction of low, medium, or high quality of a given wine, based on fixed acidity, volatile acidity, 
chlorides, and alcohol content.


## Show confusion matrix

In [39]:
from sklearn.metrics import confusion_matrix
display("Confusion matrix:")
display(confusion_matrix(ww_quality_test, pred_model))

'Confusion matrix:'

array([[162, 185,   0],
       [ 71, 510,   0],
       [  5,  47,   0]], dtype=int64)

In this case, the confusion matrix shows that the model never predicted high quality (third column)

## Create a function with several ML models at once

In [46]:
def models(X_train, y_train, X_test, y_test):
    
    from sklearn.linear_model import LogisticRegression
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(X_train, y_train)
    log_reg_train_predictions = model.predict(X_train)
    log_reg_test_predictions = model.predict(X_test)


    print('Logistic Regression Training Accuracy:', ((accuracy_score(y_train, log_reg_train_predictions)) * 100).round(2))
    print('Logistic Regression Test Accuracy:', ((accuracy_score(y_test, log_reg_test_predictions)) * 100).round(2))

Call the big function on white wine features (input) quality (output)

In [47]:
models(ww_features_train_scaled, ww_quality_train, ww_features_test_scaled, ww_quality_test)

Logistic Regression Training Accuracy: 70.04
Logistic Regression Test Accuracy: 71.84
